In [0]:
import os
import re
import itertools  
import random
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix
from collections import defaultdict
root='/content/drive/My Drive/Colab Notebooks/docs'
len_of_shingles=10

**This is the whole story:**

![Locality-sensitive hashing](https://dl.dropboxusercontent.com/s/09xk0f07pw45qh2/mystory.png)





In [0]:


k_shingles=lambda x: len(x)>=2 # Check the length of a string (shingles)

def shinglizer(txt, n): 
  txt=re.sub(r'[^\w\s\d]',' ', txt).lower() #Replace all non-numerical and non-alphabetical characters with space
  txt=re.sub(r'\s{1,}',' ', txt) #Replace one or more spaces with one space
  txt=[(txt[i:i+n]) for i in range(0, len(txt))] #Split the input string into shingles of n characters
  txt=set(filter(k_shingles,txt)) #filter very short shingles
  return txt 

**Converting Documents To Sets of Shingles**

In [0]:
#Reading all files and their contents and then, split texts into shingles
%%time

shingles=set()

documents=os.listdir(root)
documents = random.sample(documents, len(documents)) #shuffling documents 

content=dict()
for doc in documents:
  path= os.path.join(root, doc)
  file = open(path, 'r')
  txt=file.read()
  shn=shinglizer(txt,len_of_shingles)
  content[doc]=shn
  shingles.update(shn)

  file.close()

shingles=list(shingles)
shingles = random.sample(shingles, len(shingles)) #shuffling shingles 

CPU times: user 174 ms, sys: 69.6 ms, total: 243 ms
Wall time: 531 ms


**Generating Shingles-Documents Matrix**

In [0]:
#Generating shingles-documents matrix
%%time
mtx=pd.DataFrame(0,columns=documents,index=shingles)

for doc in documents:
  for shingle in content[doc]:
    mtx.at[shingle,doc]=1

print('The shape of the mtx is: ', mtx.shape)

The shape of the mtx is:  (34583, 436)
CPU times: user 976 ms, sys: 27.3 ms, total: 1 s
Wall time: 1.01 s


**Generating Signature Matrix**

In [0]:
#Let's assume there are 100 hash functuions to produce signatures for all sets(documents)
%%time

denominator=mtx.shape[0]
permutation=lambda a,x,c : (a*x+c) % denominator 

signature_length=100 #Num of hash functions as well.


def minhash(a,c,li): #It hashes the indices and return the minimum index
  hashed_list=[permutation(a,x,c) for x in li]
  return min(hashed_list)

def signature(mtx):
  c=random.sample(list(range(1,1001)),k=1000)
  a=random.sample(list(range(1,1001)),k=1000)
  
  lil=lil_matrix(mtx.values) #Sparse representation Help to speed up computation and save resources
  signs=pd.DataFrame('-',columns=documents, index=list(range(0,signature_length)))

  for i in range(0,lil.get_shape()[1]): #iterating over documents
    coli_nz_idx=lil.getcol(i).nonzero()[0] #return a numpy array of indices of nonzero elements of column i
    signs[documents[i]]=[minhash(a[j],c[j],coli_nz_idx) for j in range(0, signature_length)]  
  return signs

sig=signature(mtx)

CPU times: user 54.3 s, sys: 30.8 ms, total: 54.3 s
Wall time: 54.4 s


**Cosine-based L.S.H. implementation**

The figure shows what happens in each iteration.

![Cosine-based L.S.H. implementation](https://dl.dropboxusercontent.com/s/ilxxafrt83fusr2/cosines.png)




In [0]:
# Cosine-based L.S.H. implementation
%%time

r=10 #num_of_rows_in_a_bar
b=signature_length/r #num_of_bars (The num of hash sig. hash fucts is 100)
s=(1/b)**(1/r) #Threshold or the probability_of_candidacy 
print("The Threshold for the candidacy is %5.2f" % s)

distance=lambda U,v: [np.dot(u,v) for u in U]


def randvect(r): #Returns a random list of -1 and 1
  rv=np.random.randint(2, size=r)
  replace=lambda x: x-1 if(x==0) else x
  rv=list(map(replace,rv))
  return rv

def lsh_cosine(sig):

  v=randvect(r)

  bars=np.array(pd.DataFrame(sig, index=list(range(0,r)),columns=documents)).T
  bar_cos=distance(bars,v)
  bar_cos=pd.DataFrame(bar_cos,columns=[0],index=documents)
  cosines=pd.DataFrame(bar_cos)

  for i in range (r,signature_length,r):
    #v=randvect(r)
    bars=np.array(pd.DataFrame(sig, index=list(range(i,i+r)),columns=documents)).T
    bar_cos=distance(bars,v)
    bar_cos=pd.DataFrame(bar_cos,columns=[int(i/r)],index=documents)
    cosines=pd.concat([cosines, bar_cos], axis=1)
  return cosines,v
  
cosines,vector=lsh_cosine(sig)
#print(vector)


**Updating bins with an OR Construction**

![Updating bins with an OR Construction](https://dl.dropboxusercontent.com/s/6ilnejffw2z777n/bins.png)



In [0]:
%%time
noSameElements=lambda t: t[0]!=t[1]

def assign_a_binID(li):
  bits = [str(i) for i in li]
  bits = "".join(bits)
  return int(bits,2)

#Converting signs to booleans and then, bits
bins=np.array(cosines.ge(0))
bins=np.multiply(bins, 1) 
binsDict=defaultdict(list)


for i,doc in enumerate(documents):
  binID=assign_a_binID(bins[i])
  binsDict[binID].append(doc)

candidate_pairs=set()
for i in binsDict.keys():
  if (len(binsDict[i])>=2):
    pairs=set(itertools.product(binsDict[i],binsDict[i]))
    pairs=set([tuple(sorted(item)) for item in pairs])
    pairs=set(filter(noSameElements,pairs))
    candidate_pairs.update(pairs)
    

**Validation: L.S.H vs. Jaccard**

In [0]:
#Brute Force comaprison
brute_pairs=[]
jaccardWithShingles=lambda d : len(content[d[0]].intersection(content[d[1]])) / float(len(content[d[0]].union(content[d[1]])))

for d1 in documents:
  for d2 in documents:
    if(d1!=d2):
      jws=jaccardWithShingles((d1,d2))
      if((jws>=0.78) and ((d2,d1) not in brute_pairs)):
        print('(%s , %s) = %f' %(d1,d2,jws))
        brute_pairs.append((d1,d2))  


In [0]:
cart=len(documents)**2
cart=(cart-len(documents))/2
p=len(candidate_pairs)/cart * 100
print('Almost %5.2f percent of document-pairs are candidates to be compared.'% p)
print('And the number of candidate pairs are %d, including: '% len(candidate_pairs))


Almost  0.16 percent of document-pairs are candidates to be compared.
And the number of candidate pairs are 155, including: 


In [0]:
print(candidate_pairs)

In [0]:
#L.S.H
%%time
lsh_pairs=[]
for pair in candidate_pairs:
  jws=jaccardWithShingles(pair)
  if(jws>=0.78):
    print('(%s , %s) = %f' %(pair[0],pair[1],jws))
    lsh_pairs.append(pair)
print('The number of False Positives is %d' %(len(candidate_pairs)-len(lsh_pairs)))
print('The number of False Negatives is %d' %(len(brute_pairs)-len(lsh_pairs)))